In [15]:
import xgboost as xgb
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras

In [16]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx"
data = pd.read_excel(url)

print(data.head())
# print(data.info())
# print(data.describe())

     X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2
0  0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33
1  0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33
2  0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33
3  0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33
4  0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28


In [17]:
X = data.iloc[:, :-2]
y = data['Y1']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost Regressor": xgb.XGBRegressor(n_estimators=100, random_state=42)
}

In [19]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")
mlflow.set_experiment("heat_load_prediction")
model_name = "heat-forecast-prod"

for name, model in models.items():
    with mlflow.start_run():

        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        mlflow.log_param("model_name", name)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.sklearn.log_model(model, name = model_name, input_example = X_train_scaled)

2025/12/07 11:01:12 INFO mlflow.tracking.fluent: Experiment with name 'heat_load_prediction' does not exist. Creating a new experiment.
2025/12/07 11:01:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run fearless-robin-603 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/f69907b0cae44f38a85a21793b487d1b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


2025/12/07 11:01:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run bouncy-fawn-187 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/994d0e6b25054671b22916f8f2bc9553
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


2025/12/07 11:01:27 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run invincible-hawk-713 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/411c2e26d3fa40dbace20764491fe2b6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


In [26]:
model = keras.Sequential([
    keras.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1)   # regression output
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="mse"
)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

with mlflow.start_run():
    model.fit(X_train_scaled, y_train, epochs=500, batch_size=32, validation_split=0.2, verbose = 1, callbacks=[early_stopping])
    y_pred = model.predict(X_test_scaled)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_param("model_name", "Neural Networks")
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)
    mlflow.sklearn.log_model(model, name = model_name, input_example = X_train_scaled)

Epoch 1/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 592.3196 - val_loss: 511.3568
Epoch 2/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 550.3879 - val_loss: 469.2422
Epoch 3/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 500.2742 - val_loss: 412.5868
Epoch 4/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 429.1485 - val_loss: 333.4996
Epoch 5/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 334.0253 - val_loss: 235.7348
Epoch 6/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 221.6455 - val_loss: 140.1665
Epoch 7/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 123.2368 - val_loss: 72.7925
Epoch 8/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 64.4937 - val_loss: 48.0010
Epoch 9/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 45.4173 - val_loss: 41.9285
Epoch 10/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 38.8512 - val_loss: 35.6661
Epoch 11/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 33.7701 - val_loss: 32.1511
Epoch 12/500
16/16 ━━━━━━━

2025/12/07 11:16:22 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
🏃 View run skittish-wasp-64 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/97961190ba4f4580a17d5f9265fda485
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


In [24]:
# Register model
run_id = input('Run ID:')
model_uri = f'runs:/{run_id}/{model_name}'
with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Run ID: 411c2e26d3fa40dbace20764491fe2b6


Successfully registered model 'heat-forecast-prod'.
2025/12/07 11:09:14 WARNING mlflow.tracking._model_registry.fluent: Run with id 411c2e26d3fa40dbace20764491fe2b6 has no artifacts at artifact path 'heat-forecast-prod', registering model based on models:/m-98fa2ef0ec1142128ca25f3969ab9962 instead
2025/12/07 11:09:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: heat-forecast-prod, version 1
Created version '1' of model 'heat-forecast-prod'.


🏃 View run invincible-hawk-713 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/411c2e26d3fa40dbace20764491fe2b6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


In [25]:
model_uri = f"models:/{model_name}@champion"
loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test_scaled)
print(y_pred[:4])
y_test[:4]

[15.09619  13.24421  32.464813 41.37811 ]


668    16.47
324    13.17
624    32.82
690    41.32
Name: Y1, dtype: float64